# Mushroom Dataset

## Librerie

In [38]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sl

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

print(tf.__version__)
print(pd.__version__)
print(np.__version__)
print(sl.__version__)

2.2.0
1.0.4
1.18.4
0.22.2.post1


## Download del dataset

Importiamo il dataset dalla [repository GitHub](https://github.com/rirolli/Mushroom/blob/master/mushroom_data_all.csv) in cui è stato salvato:

In [0]:
url = 'https://raw.githubusercontent.com/rirolli/Mushroom/master/mushroom_data_all.csv'

mushroom_data = pd.read_csv(url)

In [40]:
# Stampa delle prime 5 righe del dataset
mushroom_data.head()

,class_edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


### Ottimizzazione del dataset

Preparazione dei dati rendendoli tutti in forma numerica in quanto il modello della rete neurale di tensorflow esegue calcoli solo su numeri e non sulle stringhe:

In [0]:
for elem in mushroom_data:
  mushroom_data[elem] = (pd.Categorical(mushroom_data[elem])).codes

In [42]:
# Stampa delle prime 5 righe del dataset codificato
mushroom_data.head()

,class_edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


### Ripartizione del dataset

Si esegue lo Split del dataset per ricavare i dati di test e di valutazione: 

In [0]:
# Il dataset viene diviso il due parti:
# y che contiene solo la colonna 'class_edible'
# X che contiene tutte le altre colonne.
y = mushroom_data.class_edible
X = mushroom_data.drop(labels=['class_edible'],axis=1)

# Il dataset viene nuovamente diviso in altre parti:
# train_X e train_y che sono le parti usate per l'addestramento;
# val_X e val_y che contengono le parti per la valutazione della rete;
# test_X e test_y per effettuare le predizioni e assicurarsi il corretto
# funzionamento. 
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=.2)
train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=.1)

# I dati vengono poi memorizzati dentro un Dataset di Tensorflow per sfruttarli
# per l'addestramento. Questa stessa operazione viene poi fatta anche per il
# test set e per l'evaluetion set.
train_dataset = (tf.data.Dataset.from_tensor_slices((train_X.values, train_y.values))).shuffle(len(train_X)).batch(1)
val_dataset = (tf.data.Dataset.from_tensor_slices((val_X.values, val_y.values))).shuffle(len(val_X)).batch(1)
test_dataset = (tf.data.Dataset.from_tensor_slices(test_X.values)).batch(1)

## La rete neurale

Il modello utilizzato è formato da 3 layer totalmente connessi e viene utilizzato come algoritmo di ottimizzazione l'adam:

In [0]:
num_inputs = X.shape[1]

In [45]:
model = keras.Sequential([
  tf.keras.layers.Dense(128, input_dim=num_inputs, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               2944      
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 19,585
Trainable params: 19,585
Non-trainable params: 0
_________________________________________________________________


## Addestramento della rete neurale

Ora è il momento di addestrare il modello tramite i dati di train:

In [46]:
# Addestramento della rete neurale
model.fit(train_dataset, epochs=30)

Epoch 1/30
5849/5849 [==============================] - 8s 1ms/step - loss: 0.1217 - accuracy: 0.9513
Epoch 2/30
5849/5849 [==============================] - 8s 1ms/step - loss: 0.0256 - accuracy: 0.9904
Epoch 3/30
5849/5849 [==============================] - 9s 1ms/step - loss: 0.0160 - accuracy: 0.9938
Epoch 4/30
5849/5849 [==============================] - 9s 2ms/step - loss: 0.0194 - accuracy: 0.9940
Epoch 5/30
5849/5849 [==============================] - 10s 2ms/step - loss: 0.0187 - accuracy: 0.9952
Epoch 6/30
5849/5849 [==============================] - 9s 1ms/step - loss: 0.0157 - accuracy: 0.9969
Epoch 7/30
5849/5849 [==============================] - 8s 1ms/step - loss: 0.0086 - accuracy: 0.9979
Epoch 8/30
5849/5849 [==============================] - 8s 1ms/step - loss: 0.0094 - accuracy: 0.9974
Epoch 9/30
5849/5849 [==============================] - 9s 1ms/step - loss: 0.0122 - accuracy: 0.9969
Epoch 10/30
5849/5849 [==============================] - 8s 1ms/step - loss: 0.01

## Valutazione della rete neurale

Eseguiamo la valutazione del modello:

In [47]:
# Valutazione del modello
val_loss, val_acc = model.evaluate(val_dataset)
print("\nTest accuracy: {:.2f} ({:.2%})".format(val_acc, val_acc))

1625/1625 [==============================] - 2s 1ms/step - loss: 3.2617e-06 - accuracy: 1.0000

Test accuracy: 1.00 (100.00%)


## Predizione tramite la rete neurale

Ora è il momento di predire i valori del dataset di test tramite l'addestramento appena effettuato:

In [0]:
# Predizione dei dati tramite il modello
predictions = model.predict(test_dataset)

Effettuiamo una breve stampa dei valori ottenuti tramite `predict` e degli effettivi valori di `val_y`. Qui vengono mostrato anche il valore di accuratezza della predizione:

In [49]:
r=0
f=0

# Eseguiamo la stampa dei valori 'predictions' ottenuti tramite
# la predizione con la rete neurale e delle etichette reali 'val_y'.
print("Label \t Prediction \t Description\n")
for test, pred in zip(test_y, tf.sigmoid(predictions).numpy()):
  message = "{}\t{:.2%}".format(test, pred[0])
  if test==np.round(pred[0]):
    r+=1
  else:
    message += "\t PREDIZIONE ERRATA"
    f+=1
  print(message)

print(f"\n - Totale predizioni corrette: {r}\n - Totale predizioni errate: {f}\n - Accuratezza predizione: {(r/len(predictions_rounded))*100}")

Label 	 Prediction 	 Description

1	100.00%
0	0.00%
1	100.00%
0	0.00%
0	0.00%
0	0.00%
1	100.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
1	100.00%
1	100.00%
1	100.00%
1	100.00%
0	0.00%
1	100.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
0	0.00%
0	0.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
0	0.00%
0	0.00%
1	100.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
1	100.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
1	100.00%
1	100.00%
1	100.00%
1	100.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
1	100.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
1	100.00%
1	100.00%
0	0.00%
1	100.00%
1	100.00%
0	0.00%
1	100.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.00%
0	0.00%
1	100.00%
0	0.00%
1	100.00%
0	0.

## Confusion Matrix

Tramite i valori ottenuti è possibile generare una `confuion_matrix`:

In [0]:
predictions_rounded = tf.round(tf.sigmoid(predictions)).numpy()

In [51]:
print(confusion_matrix(test_y, predictions_rounded, normalize=None))

[[320   0]
 [  0 330]]
